1. generic lessons by day 
2. new cohort start stop
3. updated holidays

In [2]:
from csv_ical import Convert
import pandas as pd
import numpy as np
import re


In [3]:
# Create dataframe of student holidays
holidays_df = pd.read_csv('student_holidays.csv')
holidays_df['Date'] = pd.to_datetime(holidays_df.Date)
holidays_df.index = pd.to_datetime(holidays_df.Date)

In [4]:
# Create dataframe of Cohorts with Name, Start and End Dates
labels = ['cohort', 'start_date', 'end_date']
easley = ['easley',  '12/07/2020', '06/11/2021']
florence = ['florence', '03/15/2021', '09/03/2021']
germain = ['germain', '06/14/2021', '12/07/2021']
hopper = ['hopper', '09/07/2021', '03/14/2022']
innis = ['innis', '12/13/2021', '06/16/2022']

cohorts_start_stop = pd.DataFrame([easley, florence, germain, hopper, innis])
cohorts_start_stop.columns = labels

new_cohort = 'innis'
new_cohort_start_end = cohorts_start_stop[cohorts_start_stop.cohort == new_cohort]
new_cohort_start_end

,cohort,start_date,end_date
4,innis,12/13/2021,06/16/2022


In [5]:
# df with start and end
new_cohort_days = pd.DataFrame({'date': [new_cohort_start_end.start_date.iloc[0], new_cohort_start_end.end_date.iloc[0]]})

#convert to datetime
new_cohort_days = pd.DataFrame(pd.to_datetime(new_cohort_days.date)).set_index(new_cohort_days.date).asfreq('D')

# join on holidays dataframe
new_cohort_days = new_cohort_days.join(holidays_df, how='left').drop(columns=['date', 'Date'])

# get weekday
new_cohort_days['weekday'] = new_cohort_days.index.day_name()

# label weekends
new_cohort_days.loc[new_cohort_days.weekday == 'Saturday', 'Student Holidays'] = 'Weekend'
new_cohort_days.loc[new_cohort_days.weekday == 'Sunday', 'Student Holidays'] = 'Weekend'

# new column names
new_cohort_days.columns = ['student_holiday', 'weekday']

new_cohort_days['is_classday'] = new_cohort_days.student_holiday.isnull().astype(int)

In [6]:
# create hours dataframe that has default class hours by day of week. 
hours = pd.DataFrame({'weekday': ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'],
                    'is_classday': [1, 1, 1, 1, 1, 0, 0], 'class_hours': [6, 7, 3.5, 6.5, 7, 0, 0]})

In [7]:
# Merge cohort dataframe with the hours dataframe. 
new_cohort_daily_hours = new_cohort_days.merge(hours, how='left', on=['weekday', 'is_classday'])
# give new dataframe the original date index. 
new_cohort_daily_hours.index = new_cohort_days.index

In [8]:
# get classday number 
new_cohort_daily_hours['classday_num'] = np.arange(new_cohort_daily_hours.shape[0])

new_cohort_daily_hours = pd.concat([
    new_cohort_daily_hours[new_cohort_daily_hours.is_classday == 1].classday_num.rank().rename('classday_num2'),
    new_cohort_daily_hours
], axis=1).drop(columns='classday_num').rename(columns={'classday_num2': 'classday_num'})

In [9]:
# read csv with generic lesson names by class day number taught
generic_lesson_template = pd.read_csv('generic_lessons_days.csv')

generic_lesson_template = generic_lesson_template[['classday_num', 'generic_event_name']]

In [10]:
new_cohort_daily_hours = new_cohort_daily_hours.reset_index()

In [ ]:
classdays = new_cohort_daily_hours[new_cohort_daily_hours.is_classday==1].set_index('classday_num').drop(columns=['student_holiday', 'is_classday'])

lessons_df = generic_lesson_template[generic_lesson_template.classday_num.notnull()].set_index('classday_num')

new_cal = lessons_df.join(classdays, how='left')

new_cal['start'] = new_cal['index']

new_cal = new_cal[['generic_event_name', 'index']]
new_cal.columns = ['Subject', 'Start Date']

new_cal.to_csv('innis_cal.csv')